### Middleware Function
Accepts a request, response objects and a next function. Can be chained together such that the request flows through the chain. Middleware can be application level or router level.

**Application Level** all requests to the application (or to the specified route) will pass through the middleware (unless chain is ended earlier) . To use application at application level:

In [ ]:
// timeLog is a middleware
function timeLog(req, res, next) {
  console.log('Time:', Date.now())
  next()
}

app.use(timeLog)

In [ ]:
// app.use works for any HTTP request method
app.use('/user/:id', function (req, res, next) {
  console.log('Request Type:', req.method)
  next()
})

**Router Level:** similar to application level

In [ ]:
const express = require('express')
const app = express()
const router = express.Router()

// This code is executed for every request to the router
router.use(function (req, res, next) {
  console.log('Time:', Date.now())
  next()
})

### Middleware Sequence
Consider the below setup:

In [ ]:
const express = require('express')
const app = express()

// First middleware
app.use(function (req, res, next) {
    console.log(`A. Entry Point, ${req.method}`)
    next()
})

// Second middleware
app.use(function (req, res, next) {
    console.log(`B. Request time: ${Date.now()}, ${req.method}`)
    next()
})

app.get('/info', function (req, res, next) {
    console.log('C. First handler for /info')
    res.send('Welcome')
})

app.get('/info', function (req, res, next) {
    console.log('D. Second handler for /info')
    res.send('Again')
})

app.use(function (req, res, next) {
    console.log(`E. End at ${Date.now()}, ${req.method}`)
})

app.listen(3000, function () {
    console.log('Server started')
})


If a user request /info, the console will log:
```
A. Entry Point, GET
B. Request time: 1602006984556, GET
C. First handler for /info
```

The second handler for /info is not even called. Now consider the change made to the first handler:

In [ ]:
app.get('/info', function (req, res, next) {
    console.log('C. First handler for /info')
    res.send('Welcome')
    next()
})

In this case, the console will log:
```
A. Entry Point, GET
B. Request time: 1602006984556, GET
C. First handler for /info
D. Second handler for /info
Error [ERR_HTTP_HEADERS_SENT]: Cannot set headers after they are sent to the client
```

The above output indicates that the second handler was invoked but we got an error (because once a response has been sent we cannot resend response). So we make the following change in the first handler:

In [ ]:
app.get('/info', function (req, res, next) {
    console.log('C. First handler for /info')
    res.status('201')
    next()
})

app.get('/info', function (req, res, next) {
    console.log('D. Second handler for /info')
    res.send('Welcome Again')
})

This time we don't get any error and the client gets the HTML with 'Welcome Again' as text. However the last middleware is not executed. In order to do so, we make the following change to the second handler

In [ ]:
app.get('/info', function (req, res, next) {
    console.log('D. Second handler for /info')
    res.send('Welcome Again')
    next()
})

`next` actually works a bit like *GOTO*, call returns back to the point after the next call and execution continues from that point. So we can modify the code to the following:

In [ ]:
const express = require('express')
const app = express()

// First middleware
app.use(function (req, res, next) {
    console.log(`A. Entry Point, ${req.method}`)
    next()
    console.log(`E. End at ${Date.now()}, ${req.method}`)
})

// Second middleware
app.use(function (req, res, next) {
    console.log(`B. Request time: ${Date.now()}, ${req.method}`)
    next()
})

app.get('/info', function (req, res, next) {
    console.log('C. First handler for /info')
    res.status('201')
    next()
})

app.get('/info', function (req, res) {
    console.log('D. Second handler for /info')
    res.send('Welcome Again')
})

app.listen(3000, function () {
    console.log('Server started')
})

If we visit /info, in console we get:

```
A. Entry Point, GET
B. Request time: 1602008894050, GET
C. First handler for /info
D. Second handler for /info
E. End at 1602008894108, GET
```